In [17]:
import pandas, os
import geopy
from geopy.geocoders import Nominatim
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString
from datetime import datetime
import folium
from folium import plugins
import numpy as np
import seaborn as sns



gps = pd.read_csv('elenco_seggi_gps.csv')
p13 = pd.read_csv('pisa_2013.csv')
p18 = pd.read_csv('pisa_2018.csv')


#############QUESTI CAZZO DI DUE VALORI ANDRANNO DECISI DAL MENU

part='ucic'
anno=18

#########################################
delta = 0.001

noise = [gps['lat'][i] + np.random.uniform(-delta,delta) for i in range(len(gps))]
gps['lat_n']=noise
noise = [gps['lon'][i] + np.random.uniform(-delta,delta) for i in range(len(gps))]
gps['lon_n']=noise



vet=sns.color_palette("GnBu_r", 100)
vet = [vet[99-i] for i in range(100)]

def RGB(r,g,b): 
    R=int(255*r)
    G=int(255*g)
    B=int(255*b)
    return '#%02x%02x%02x' % (R,G,B)

m = folium.Map([43.7159395 , 10.4158624], tiles='Stamen Terrain',
               control_scale = True, zoom_start=13)


if (anno==13):
        data=p13
else:
        data=p18
        
        
nseggio = gps['num'].values.tolist()
percpart = [data[part][i]/data['tot'][i]*100 for i in range(len(gps))]
labels = [nseggio[i]+'xxx'+part.title()+': '+'%.2f' % percpart[i]+'%' for i in range(len(gps))]

for i in range(len(gps)):

    k=int((data[part][i]/data['tot'][i])*99*99/60)
    col=RGB(vet[k][0],vet[k][1],vet[k][2])
    popup = folium.Popup(labels[i], parse_html=True)
    folium.CircleMarker(location=[gps['lat_n'][i], gps['lon_n'][i]],
                                  radius=20, color='black', fill=True, popup=popup,
                                  fill_opacity=0.75, fill_color=col).add_to(m)
              

m.save('perc'+str(anno)+'_'+part+'.html')


import plotly.plotly as py
import plotly.graph_objs as go
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
print (__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)
import plotly.graph_objs as go


graph = go.Bar(
            x=[i+1 for i in range(86)],
            y=percpart,
    #name='Stazione F.S.',
    opacity=0.75,
    width=0.75, 
    marker=dict(
        color=[RGB(vet[int((data[part][i]/data['tot'][i])*99*99/60)][0],
     vet[int((data[part][i]/data['tot'][i])*99*99/60)][1],
     vet[int((data[part][i]/data['tot'][i])*99*99/60)][2]) for i in range(len(gps))],
        line=dict(
            color='black',
            width=0.5,)) )
data =[graph]

layout = go.Layout(
    title='Voti in percentuale per sezione - ' + part.title() + ' 20' + str(anno),
    xaxis=dict(
        title='Sezione n.',
        titlefont=dict(
           # family='Sans serif',
            size=18,
            #color='#000000'
        )
    ),
    yaxis=dict(
        title='Voti %',
        titlefont=dict(
            #family='Courier New, monospace',
            size=18,
            #color='#000000'
        )
    )
)

fig = go.Figure(data = data, layout = layout)
plot_url = plot(fig, filename='graph_perc'+str(anno)+'_'+part+'.html')

2.7.0
